In [2]:
rm(list=ls())

# apt 가격 데이터 셋
### setwd 
# setwd("C:/Users/kdg95/Documents")

### packages
library(caret)
library(e1071)
library(dplyr)
library(mice)
# install.packages("lime"); 
# library(lime)
# install.packages("mice"); 

In [3]:
### data 불러오기
dat = read.csv("data/dat.csv")
head(dat)
str(dat)
dat$is_cancer = as.factor(dat$is_cancer)
dat <- dat[-1]

X,mean.radius,mean.texture,mean.perimeter,mean.area,mean.smoothness,mean.compactness,mean.concavity,mean.concave.points,mean.symmetry,...,worst.texture,worst.perimeter,worst.area,worst.smoothness,worst.compactness,worst.concavity,worst.concave.points,worst.symmetry,worst.fractal.dimension,is_cancer
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0
5,12.45,15.70,82.57,477.1,0.12780,0.17000,0.1578,0.08089,0.2087,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,0


'data.frame':	569 obs. of  32 variables:
 $ X                      : int  0 1 2 3 4 5 6 7 8 9 ...
 $ mean.radius            : num  18 20.6 19.7 11.4 20.3 ...
 $ mean.texture           : num  10.4 17.8 21.2 20.4 14.3 ...
 $ mean.perimeter         : num  122.8 132.9 130 77.6 135.1 ...
 $ mean.area              : num  1001 1326 1203 386 1297 ...
 $ mean.smoothness        : num  0.1184 0.0847 0.1096 0.1425 0.1003 ...
 $ mean.compactness       : num  0.2776 0.0786 0.1599 0.2839 0.1328 ...
 $ mean.concavity         : num  0.3001 0.0869 0.1974 0.2414 0.198 ...
 $ mean.concave.points    : num  0.1471 0.0702 0.1279 0.1052 0.1043 ...
 $ mean.symmetry          : num  0.242 0.181 0.207 0.26 0.181 ...
 $ mean.fractal.dimension : num  0.0787 0.0567 0.06 0.0974 0.0588 ...
 $ radius.error           : num  1.095 0.543 0.746 0.496 0.757 ...
 $ texture.error          : num  0.905 0.734 0.787 1.156 0.781 ...
 $ perimeter.error        : num  8.59 3.4 4.58 3.44 5.44 ...
 $ area.error             : num  153.

In [4]:
### train / test 분할
set.seed(1)
table(dat$is_cancer)
idx = createDataPartition(dat$is_cancer, p = 0.7, list=F)
train = dat[idx,]
test = dat[-idx,]
table(train$is_cancer)
table(test$is_cancer)
str(train)


  0   1 
212 357 


  0   1 
149 250 


  0   1 
 63 107 

'data.frame':	399 obs. of  31 variables:
 $ mean.radius            : num  20.6 19.7 11.4 18.2 13.7 ...
 $ mean.texture           : num  17.8 21.2 20.4 20 20.8 ...
 $ mean.perimeter         : num  132.9 130 77.6 119.6 90.2 ...
 $ mean.area              : num  1326 1203 386 1040 578 ...
 $ mean.smoothness        : num  0.0847 0.1096 0.1425 0.0946 0.1189 ...
 $ mean.compactness       : num  0.0786 0.1599 0.2839 0.109 0.1645 ...
 $ mean.concavity         : num  0.0869 0.1974 0.2414 0.1127 0.0937 ...
 $ mean.concave.points    : num  0.0702 0.1279 0.1052 0.074 0.0598 ...
 $ mean.symmetry          : num  0.181 0.207 0.26 0.179 0.22 ...
 $ mean.fractal.dimension : num  0.0567 0.06 0.0974 0.0574 0.0745 ...
 $ radius.error           : num  0.543 0.746 0.496 0.447 0.584 ...
 $ texture.error          : num  0.734 0.787 1.156 0.773 1.377 ...
 $ perimeter.error        : num  3.4 4.58 3.44 3.18 3.86 ...
 $ area.error             : num  74.1 94 27.2 53.9 51 ...
 $ smoothness.error       : num  0.00522

In [ ]:
### randomforest
control = trainControl(method='cv', search='random', number=5,verbose = TRUE)
rf.model <- train(
  is_cancer ~ .,
  data = train,
  tuneLength = 10,
  trControl = control,
  method="rf")


rf.grid = expand.grid(
  .mtry = c(1,3,5)
)
control = trainControl(method='cv', search='grid', number=5,verbose = TRUE)
rf.model <- train(
  is_cancer ~ .,
  data = test,
  tuneGrid = rf.grid,
  trControl = control,
  method = 'rf'
)
rf.model

pred.rf <- predict(rf.model,test[-31])
confusionMatrix(pred.rf, test[,31])

In [ ]:
### xgboost
## random 탐색
control = trainControl(method='cv', search='random', number=5,verbose = TRUE)
xgb.model <- train(
  is_cancer ~ .,
  data = train,
  tuneLength = 10,
  trControl = control,
  method="xgbTree")

xgb.model

pred.xgb <- predict(xgb.model,test[-31])
confusionMatrix(pred.xgb, test[,31])

In [ ]:
## 격자 탐색
xgb.grid = expand.grid(
  nrounds = c(300,500),
  eta = c(0.03,0.05),
  gamma = c(3,5),
  max_depth = c(4,6),
  min_child_weight = c(6,8),
  colsample_bytree = c(0.3,0.5),
  subsample = c(0.2,0.6)
)

control = trainControl(method='cv', search='grid', number=5,verbose = TRUE)
xgb.model <- train(
  is_cancer ~ .,
  data = test,
  tuneGrid = xgb.grid,
  trControl = control,
  method = 'xgbTree'
)

In [ ]:
### gbm
## random 탐색
control = trainControl(method='cv', search='random', number=2,verbose = TRUE)
gbm.model <- train(
  is_cancer ~ .,
  data = train,
  tuneLength = 3,
  trControl = control,
  method="gbm")

gbm.model$bestTune

pred.gbm <- predict(gbm.model,test[-31])
confusionMatrix(pred.gbm, test[,31])

## 격자 탐색
gbm.grid = expand.grid(
  shrinkage = c(0.1,0.3),
  interaction.depth = c(3,6,9),
  n.minobsinnode = c(5,10,15),
  n.trees = c(500,100,1500)
)

control = trainControl(method='cv', search='grid', number=5,verbose = TRUE)
gbm.model <- train(
  is_cancer ~ .,
  data = test,
  tuneGrid = gbm.grid,
  trControl = control,
  method = 'gbm'
)

pred.gbm <- predict(gbm.model,test[-31])
confusionMatrix(pred.gbm, test[,31])

In [ ]:
dim(train)
# ### lime
# #### lime ####
# explainer <- lime(train[,-31], xgb.model)
# explanation <- explain(test[1:5,-31],            # test data
#                        explainer,                # lime 적용 
#                        labels = NULL,            # 모델이 분류기일 경우 지정
#                        n_labels = 1,          # 모델이 분류기일 경우 지정
#                        n_features = 3,           # 각 설명에 사용할 기능의 수
#                        #n_permutations = 2,      # 각 설명에 사용할 순열 수
#                        feature_select = 'auto',  # auto, highest_weighs, none,
#                        # forward_selection, lasso_path, tree
#                        dist_fun = 'gower',       # 거리 함수
#                        kernel_width = NULL       # dist_fun이 gower일 경우 NULL
# )
# explanation[,1:9]
# plot_features(explanation, ncol = 2)
# ?explain
